# VoxCity Demo — Landmark Visibility (Advanced Parameters)

This notebook demonstrates `voxcity.simulator.view.get_landmark_visibility_map` with options to pick landmarks by ID, polygon, or automatically from the center point.


In [ ]:
# %pip install voxcity

from voxcity.generator import get_voxcity
from voxcity.geoprocessor.draw import draw_rectangle_map_cityname
from voxcity.simulator.view import get_landmark_visibility_map

meshsize = 5
cityname = "Tokyo, Japan"

rectangle_vertices = [
    (139.760, 35.680),  # SW
    (139.760, 35.690),  # NW
    (139.770, 35.690),  # NE
    (139.770, 35.680)   # SE
]

voxcity_grid, building_height_grid, building_min_height_grid, \
    building_id_grid, canopy_height_grid, canopy_bottom_height_grid, land_cover_grid, dem_grid, \
    building_gdf = get_voxcity(
        rectangle_vertices,
        'OpenStreetMap', 'OpenStreetMap',
        'High Resolution 1m Global Canopy Height Maps',
        'DeltaDTM',
        meshsize,
        output_dir='output/landmark_demo'
)

len(building_gdf)


## Select landmarks
- Option 1: Provide `landmark_building_ids`
- Option 2: Provide a `landmark_polygon` (GeoJSON-style)
- Option 3: Omit both and the function will pick buildings at the center point of the rectangle


In [ ]:
landmark_kwargs = {
    "view_point_height": 1.5,
    "colormap": "cool",
    "obj_export": True,
    "output_directory": "output/landmark_demo",
    "output_file_name": "landmark_visibility",
    "alpha": 1.0,
}

# Example 1: Pick by IDs (take first few as sample)
ids_sample = building_gdf.head(3).index.tolist()

landmark_vis_map, voxcity_grid_marked = get_landmark_visibility_map(
    voxcity_grid, building_id_grid, building_gdf, meshsize,
    landmark_building_ids=ids_sample,
    **landmark_kwargs
)

landmark_vis_map.shape


In [ ]:
# Example 2: Auto-select using rectangle center (omit IDs and polygon)
landmark_vis_map_auto, _ = get_landmark_visibility_map(
    voxcity_grid, building_id_grid, building_gdf, meshsize,
    rectangle_vertices=rectangle_vertices,
    **landmark_kwargs
)

landmark_vis_map_auto.shape
